## 51. N-Queens

回溯法

In [ ]:
(defun solve-n-queens (n)
  (let ((solutions))
    (loop for col below n
       do (loop for item in (find-solution 0 n `(,col))
             do  (let ((solution (loop repeat n
                                    collect (loop repeat n
                                               collect "."))))
                   (loop for col in item
                      for row below (1- n)
                      do (setf (nth col (nth row solution)) "Q")
                      finally (push solution solutions))))
       finally (return solutions))))

(defun next-positions (n &optional placed)
  "对已顺序放置 placed 的 n 皇后问题，返回下一行可能的放置位置"
  (set-difference (loop for i below n collect i)
                  (loop for col in placed
                     for i from (length placed) downto 1
                     append `(,(- col i) ,col ,(+ col i)))))

(defun find-solution (row n &optional placed)
  "placed 是已放置的皇后，第 i 个元素表示 i 行 placed[i] 列放置了皇后，
   row 是当前行，placed[row] == placed[-1]。
   对于前面行已放置元素为 placed（长度为 row+1）的 n 皇后问题，找出所有解"
  (cond
    ((= row (- n 1))
     `(,placed))
    (t
     (loop for p in (next-positions n placed)
        append (find-solution
                (1+ row) n (append placed `(,p)))))))

## 52. N-Queens II

In [ ]:
(defun total-n-queens (n)
  (length
    (loop for col below n
       append (loop for item in (find-solution 0 n `(,col))
             collect item))))

(defun next-positions (n &optional placed)
  "对已顺序放置 placed 的 n 皇后问题，返回下一行可能的放置位置"
  (set-difference (loop for i below n collect i)
                  (loop for col in placed
                     for i from (length placed) downto 1
                     append `(,(- col i) ,col ,(+ col i)))))

(defun find-solution (row n &optional placed)
  "placed 是已放置的皇后，第 i 个元素表示 i 行 placed[i] 列放置了皇后，
   row 是当前行，placed[row] == placed[-1]。
   对于前面行已放置元素为 placed（长度为 row+1）的 n 皇后问题，找出所有解"
  (cond
    ((= row (- n 1))
     `(,placed))
    (t
     (loop for p in (next-positions n placed)
        append (find-solution
                (1+ row) n (append placed `(,p)))))))

## 53. Maximum Subarray

In [ ]:
(defun max-sub-array (nums)
  (let* ((lo 0)
         (hi (1- (length nums)))
         (sum (apply #'+ nums))
         (max sum))
    (if (> (apply #'max nums) max)
        (setf max (apply #'max nums)))
    (loop while (< lo hi)
       do (let* ((n-lo (next nums lo 1))
                 (sum1 (apply #'+ (subseq nums lo (1+ n-lo))))
                 (n-hi (next nums hi -1))
                 (sum2 (apply #'+ (subseq nums n-hi (1+ hi)))))
            (if (< sum1 sum2)
                (setf lo (1+ n-lo) sum (- sum sum1))
                (setf hi (1- n-hi) sum (- sum sum2)))
            (if (> sum max)
                (setf max sum)))
       finally (return max))))

(defun next (nums start step)
  "获取从 start 位置开始，第一个负数开始的连续负数序列中的最后一个的位置"
  (loop while (< -1 start (length nums))
     do (if (>= (nth start nums) 0)
            (incf start step)
            (return (loop while (and (< 0 (+ start step) (length nums))
                                     (< (nth (incf start step) nums) 0))
                       finally (return (- start step)))))
     finally (return (- start step))))

;;; Another amazing way
;;; https://leetcode.com/problems/maximum-subarray/discuss/20396/Easy-Python-Way
(defun max-sub-array (nums)
  (loop for i from 1 below (length nums)
     do (if (plusp (nth (1- i) nums))
            (incf (nth i nums) (nth (1- i) nums)))
     finally (return (apply #'max nums))))

## 54. Spiral Matrix

找规律。

假设开始点为 (x, y)，结束点为 (m, n)。

1. 向右：(x, y) -> (x, n)
2. 向下：(x+1, n) -> (m, n)
3. 向左：(m, n) -> (m, y)
4. 向上：(m, y) -> (x+1, y)
5. 汇聚值，x、y 加一，m、n 减一，重复步骤 1-5

In [ ]:
(defun spiral-order (matrix)
  (let ((m (1- (length matrix)))
        (n (1- (length (first matrix))))
        (x 0)
        (y 0))
    (loop while (and (<= x m)
                     (<= y n))
       append (append (loop for i from y to n
                         collect (nth i (nth x matrix)))
                      (loop for i from (1+ x) to m
                         collect (nth n (nth i matrix)))
                      (loop for i from (decf n) downto y
                         when (> m x)
                         collect (nth i (nth m matrix)))
                      (loop for i from (decf m) above x
                         collect (nth y (nth i matrix))))
       do (incf x)
         (incf y))))

## 55. Jump Game

跟之前做过的一道题类似，用表存储某个位置是否能到达最后。

$$canjump(0) = {\exists} \text{canjump(i)}, {i \le nums[0]} $$

In [ ]:
(defun can-jump (nums)
  (if (null nums) (return-from can-jump nil))
  (let* ((n (length nums))
         (table (loop repeat n collect nil)))
    (setf (nth (1- n) table) t)
    (loop for i from (- n 2) downto 0
       do (loop for j from 1 to (nth i nums)
             when (and (< (+ i j) n)
                       (nth (+ i j) table))
             do (return (setf (nth i table) t))))
    (first table)))

## 56. Merge Intervals

In [ ]:
(defun merge-i (intervals)
  (let* ((intervals (sort intervals #'(lambda (a b) (< (first a)
                                                       (first b)))))
         (res `(,(car intervals)))
         (intervals (cdr intervals)))
    (loop for interval in intervals
       do (if (>= (second (car (last res)))
                  (first interval))
              (setf (second (car (last res))) 
                    (max (second (first intervals))
                         (second (car (last res)))))
              (push interval (cdr (last res))))
       finally (return res))))

## 57. Insert Interval

In [ ]:
(defun insert-i (intervals new-interval)
  (let ((res `(nil)))
    (if (null intervals)
        (return-from insert-i `(,new-interval)))
    (loop for interval in intervals
       do (cond
            (new-interval
             (if (>= (second interval)
                     (first new-interval))
                 (progn
                   (push `(,(min (first interval) (first new-interval))
                            ,(max (second interval) (second new-interval)))
                         (cdr (last res)))
                   (setf new-interval nil))
                 (push interval (cdr (last res)))))
            (t
             (if (>= (second (car (last res)))
                     (first interval))
                 (setf (second (car (last res))) 
                       (max (second (first intervals))
                            (second (car (last res)))))
                 (push interval (cdr (last res))))))
       finally (return (cdr res)))))

## 58. Length of Last Word

In [ ]:
;; 使用正则
(ql:quickload "cl-ppcre")
(defun length-of-lastword (s)
  (length (car (last (ppcre:split " " s)))))

;; 使用循环
(defun length-of-lastword (s)
  (loop for c across (reverse (string-trim " " s))
     with res = 0
     do (if (string= " " c)
            (return res)
            (incf res))
     finally (return res)))

## 59. Spiral Matrix II

In [ ]:
(defun generate-matrix (n)
  (let ((x 0)
        (y 0)
        (index 0)
        (matrix (loop repeat n
                   collect (loop repeat n collect 0)))
        (n (1- n)))
    (loop while (and (<= x n)
                     (<= y n))
       do
         (loop for i from y to n
            do (setf (nth i (nth x matrix)) (incf index)))
         (loop for i from (1+ x) to n
            do (setf (nth n (nth i matrix)) (incf index)))
         (loop for i from (1- n) downto y
            when (> n x)
            do (setf (nth i (nth n matrix)) (incf index)))
         (loop for i from (decf n) above x
            do (setf (nth y (nth i matrix)) (incf index)))
         (incf x)
         (incf y)
       finally (return matrix))))

## 60. Permutation Sequence

受启发：

> 对于 n，第一个 (n-1)! 全排列的第一位是 1，第二个 (n-1)! 全排列的第一位 2……

In [ ]:
(defun get-permutation (n k)
  (let ((numbers (loop for i from 1 to n collect i))
        (k (1- k)))
    (with-output-to-string (res)
      (loop while (>= k 0)
         do (multiple-value-bind
                  (q r)
                (floor k (reduce #'* (loop for i from 1 below (length numbers)
                                        collect i)))
              (format res "~a" (nth q numbers))
              (when (zerop r)
                (return (format res "~{~a~}" (remove (nth q numbers) numbers))))
              (setf k r)
              (setf numbers (remove (nth q numbers) numbers))))
      res)))

## 61. Rotate List

In [ ]:
(defun rotate-right (head k)
  (let* ((size (length head))
         (k (mod k size))
         (size (1- size)))
    (loop repeat k
       do (setf head (append (last head) (subseq head 0 size)))
       finally (return head))))

## 62. Unique Paths

动态规划可解。或者

> (m+n-2)! / (m-1)! / (n-1)!

查看 [Math solution, O(1) space](https://leetcode.com/problems/unique-paths/discuss/22958/Math-solution-O(1)-space)

In [ ]:
(defun unique-paths (m n)
  (let ((dp (loop repeat n
               collect (loop repeat m collect 1)))
        (m (1- m))
        (n (1- n)))
    (loop for i from 1 to n
       do (loop for j from 1 to m
             do (let ((a (if (>= (1- i) 0)
                             (nth j (nth (1- i) dp))
                             0))
                      (b (if (>= (1- j) 0)
                             (nth (1- j) (nth i dp))
                             0)))
                  (setf (nth j (nth i dp)) ; dp[i][j]
                        (+ a b))))
       finally (return (nth m (nth n dp))))))

## 63. Unique Paths II

思路跟上一题类似，但代码可以优化一下。See [Short JAVA solution](https://leetcode.com/problems/unique-paths-ii/discuss/23250/Short-JAVA-solution)。

In [ ]:
(defun unique-paths-with-obstacles (obstacle-grid)
  (let* ((m (length (first obstacle-grid)))
         (dp (loop repeat m collect 0)))
    (setf (first dp) 1)
    (loop for row in obstacle-grid
       do (loop for j to (1- m)
             do (cond
                  ((plusp (nth j row)) ; row[j] == 1
                   (setf (nth j dp) 0))
                  ((plusp j)
                   (incf (nth j dp) (nth (1- j) dp)))))
       finally (print dp) (return (nth (1- m) dp)))))

## 64. Minimum Path Sum

In [ ]:
(defun min-path-sum (grid)
  (let* ((dp (loop repeat (length (first grid)) collect 0))
         (m (length dp)))
    (loop for row in grid
       for row-idx below (length grid)
       do (loop for i below m
             do (if (zerop i)
                    (incf (nth i dp) (nth i row))
                    (if (zerop row-idx)
                        (setf (nth i dp) 
                              (+ (nth (1- i) dp)
                                 (nth i row)))
                        (setf (nth i dp)
                              (+ (nth i row)
                                 (min (nth (1- i) dp)
                                      (nth i dp)))))))
       finally (return (nth (1- m) dp)))))

## 65. Valid Number

正则表达式就是思路了。

In [ ]:
(ql:quickload "cl-ppcre")
(defun is-number (s)
  (ppcre:scan-to-strings "^\\s*[+-]?\\d*((\\d+\\.)|(\\.\\d+))\\d*(e?[-+]?\d+)\\s*$" s))

## 66. Plus One

In [ ]:
(defun plus-one (digits)
  (let ((res (copy-list digits))
        (last-idx (1- (length digits))))
    (incf (nth last-idx res))
    (loop for i from last-idx to 0
       do (if (< (nth i res) 10)
              (return res)
              (progn
                (setf (nth i res) 0)
                (if (zerop i)
                    (push 1 res)
                    (incf (nth (1- i) res)))))
       finally (return res))))

## 67. Add Binary

In [ ]:
(defun add-binary (a b)
  (let* ((size (max (length a)
                    (length b)))
         (fmt (format nil "~~~d,1,0,'0@a" size))
         (out))
    (loop for c1 across (reverse (format nil fmt a))
       for c2 across (reverse (format nil fmt b))
       with remain = 0
       do (setf remain (eval (read-from-string
                              (format nil "(+ ~a ~a ~a)" c1 c2 remain))))
         (if (> remain 1)
             (progn 
               (push (- remain 2) out)
               (setf remain 1))
             (progn
               (push  remain out)
               (setf remain 0)))
       finally (if (zerop remain)
                   (return out)
                   (progn
                     (push remain out)
                     (return (format nil "~{~a~}" out)))))))

;;; 或者更简单点
(defun add-binary (a b)
  (format nil "~b" (eval (read-from-string
                          (format nil "(+ #b~a #b~a)" a b)))))

## 68. Text Justification

In [ ]:
(defun full-justify (words max-width)
  (loop for word in words
     with res
     with line
     with width = max-width
     do (cond
          ((null line)
           (push word line)
           (decf width (length word)))
          ((>= (- width (length word) 1) 0)
           (push (format nil " ~a" word) line)
           (decf width (1+ (length word))))
          (t
           (push (justify-center (reverse line) width) res)
           (setf width (- max-width (length word)) line `(,word))))
     finally (push (format nil "~{~a~}~V@{~a~:*~}"
                           (reverse line) width " ") res)
       (return (reverse res))))

(defun justify-center (line width)
  "将一行文字 line 居中对齐，可填充剩余空格数为 width"
  (let ((interspace (1- (length line))))
    (if (zerop interspace)
        (format nil "~{~a~}~V@{~a~:*~}" line width " ")
        (progn
          (when (> width interspace)
            (setf line (loop for word in line
                          with n = (floor width interspace)
                          collect (format nil "~a~V@{~a~:*~}" word n " "))
                  width (mod width interspace))
            (setf (nth interspace line)
                  (string-right-trim " " (nth interspace line))))
          (format nil "~{~a~}" 
                  (loop for word in line
                     collect (if (>= (decf width) 0)
                                 (format nil "~a " word)
                                 word)))))))

## 69. Sqrt(x)

In [ ]:
(defun my-sqrt (x)
  (let* ((lo 1)
         (hi (floor x))
         (mid (floor (+ lo hi) 2)))
    (loop
       do (cond
            ((= (* mid mid) x)
             (return mid))
            ((> (* mid mid) x)
             (setf hi mid))
            (t
             (setf lo mid)))
         (if (= mid (floor (+ lo hi) 2))
             (return mid)
             (setf mid (floor (+ lo hi) 2))))))

## 70. Climbing Stairs

动态规划解法：f(n) = f(n-1) + f(n-2)

In [ ]:
(let ((table (make-hash-table)))
  (setf (gethash 1 table) 1)
  (setf (gethash 2 table) 2)
  (defun climb-stairs (n)
    (loop for i from 3 to n
       do (setf (gethash i table)
                (+ (gethash (1- i) table)
                   (gethash (- i 2) table)))
       finally (return (gethash n table)))))

## 71. Simplify Path

In [ ]:
(ql:quickload "split-sequence")
(defun simplify-path (path)
  (loop for p in (split-sequence:split-sequence #\/ path)
     with res
     do (cond
          ((string= ".." p)
           (pop res))
          ((string= "." p))
          ((string= "" p))
          (t
           (push p res)))
     finally (return (format nil "/~{~a~}" (reverse res)))))

## 72. Edit Distance

对于分别在 word1、word2 中的索引 i、j，`md(i, j)` 表示将 `word1[:i+1]` 转换成 `word2[:j+1]` 需要的最少步数，有：
$$md(i, j) = \begin{cases}
md(i-1, j-1) & \text(word1[i] == word2[j])\\
1 + min(md(i-1, j-1), md(i, j-1), md(i-1, j)) & \text(word1[i] != word2[j])\\
\end{cases}$$

mark 一下。

In [ ]:
(defun min-distance (word1 word2)
  (let* ((m (length word1))
         (n (length word2))
         (table (loop repeat (1+ n)
                   collect (loop repeat (1+ m) collect 0))))
    (loop for i to m
       do (setf (nth i (first table)) i))
    (loop for i to n
       do (setf (first (nth i table)) i))
    (loop for i from 1 to n
       do (loop for j from 1 to m
             do (setf (nth j (nth i table))
                      (if (string= (elt word1 (1- j))
                                   (elt word2 (1- i)))
                          (nth (1- j) (nth (1- i) table))
                          (1+ (min (nth (1- j) (nth (1- i) table))
                                   (nth (1- j) (nth i table))
                                   (nth j (nth (1- i) table)))))))
       finally (return (nth m (nth n table))))))

## 73. Set Matrix Zeroes

In [ ]:
(defun set-zeroes (matrix)
  (loop for rc in (loop for i below (length matrix)
                     append (loop for j below (length (first matrix))
                               if (zerop (nth j (nth i matrix)))
                               collect `(,i ,j)))
     do (loop for j below (length (first matrix))
           do (setf (nth j (nth (first rc) matrix)) 0))
       (loop for i below (length matrix)
          do (setf (nth (second rc) (nth i matrix)) 0))))